In [1]:
!pip install pylunar
!pip install suntime
!pip install hijri-converter
!pip install lat_lon_parser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pylunar
import datetime
import pandas as pd

from datetime import date, timedelta
from suntime import Sun, SunTimeException
from pandas import json_normalize
from hijri_converter import Hijri, Gregorian
from IPython.display import display
from lat_lon_parser import to_deg_min_sec

In [3]:
def simulateMoonPosition(longitude, latitude, year, month, day):
  _latitude = to_deg_min_sec(latitude)
  _longitude = to_deg_min_sec(longitude)

  # calculate sunset
  sun = Sun(latitude, longitude)
  abd = datetime.date(year, month, day)
  # get sunset in UTC
  abd_ss = sun.get_sunset_time(abd)
  sunset_hour = abd_ss.hour
  sunset_minute = abd_ss.minute
  sunset_second = abd_ss.second
  # YYYY, m, d, H, M, S (UTC)
  sunset = (year, month, day, sunset_hour, sunset_minute, sunset_second)

  # calculate moonrise
  mi = pylunar.MoonInfo(_latitude, _longitude)
  # calculate moonrise when sunset
  mi.update(sunset)
  altitude = mi.altitude()
  azimuth = mi.azimuth()
  age = mi.age()
  elongation = mi.elongation()

  return {
      "date": abd,
      "sunset": abd_ss,
      "moon_age": age,
      "moon_altitude": altitude,
      "moon_azimuth": azimuth,
      "moon_elongation": elongation
  }

**SIMULATION**

In [4]:
MIN_ALTITUDE_DEG = 3.0
MIN_ELONGATION_DEG = 6.4

def forecastIjtimak(longitude, latitude, startDate, endDate, monthName):
  date_range = pd.date_range(start=startDate,end=endDate)
  new_moon = None
  result = None
  json = []
  for date in date_range:
    year = date.year
    month = date.month
    day = date.day
    
    moon_position = simulateMoonPosition(longitude, latitude, year, month, day)
    altitude = moon_position['moon_altitude']
    elongation = moon_position['moon_elongation']

    condition = ((altitude >= MIN_ALTITUDE_DEG) and (elongation >= MIN_ELONGATION_DEG))
    remark = '-'
    if ((new_moon is None) and condition):
      new_moon = date
      remark = 'NEW MOON'
    
    if ((result is None) and (new_moon is not None)):
      result = date + timedelta(days=1)

    if result is not None:
      delta = date - result
      if (delta.days == 0):
        remark = monthName
    json.append({
        "date": moon_position['date'],
        "sunset": moon_position['sunset'].strftime('%H:%M'),
        "altitude": altitude,
        "elongation": elongation,
        "remark": remark
    })

  df = json_normalize(json) 
  display(df)
  return result

In [5]:
deltaTime = timedelta(days=5)
HIJRI_YEAR = 1444
latitude = -6.171697
longitude = 106.833254

# 1 RAMADHAN
ramadhan_hijri = Hijri(HIJRI_YEAR, 9, 1)
ramadhan = ramadhan_hijri.to_gregorian()
min_10_ramadhan = ramadhan - deltaTime
plus_10_ramadhan = ramadhan + deltaTime

# 1 SYAWWAL
ied_hijri = Hijri(HIJRI_YEAR, 10, 1)
ied = ied_hijri.to_gregorian()
min_10_ied = ied - deltaTime
plus_10_ied = ied + deltaTime

delta = ied - ramadhan
print('EXPECTED RAMADHAN', ramadhan_hijri, '->', ramadhan)
print('EXPECTED IED', ied_hijri, '->', ied)
print(f'Difference is {delta.days} days')

EXPECTED RAMADHAN 1444-09-01 -> 2023-03-23
EXPECTED IED 1444-10-01 -> 2023-04-21
Difference is 29 days


*Forecast First Ramadhan*

In [6]:
ijtimak_ramadhan = forecastIjtimak(longitude, latitude, min_10_ramadhan, plus_10_ramadhan, '1 Ramadhan')
print()
print('CALCULATION FOR RAMADHAN', ramadhan_hijri, '->', ijtimak_ramadhan.strftime("%Y-%m-%d"))

,date,sunset,altitude,elongation,remark
0,2023-03-18,11:05,-36.199449,314.826633,-
1,2023-03-19,11:04,-25.674126,328.534966,-
2,2023-03-20,11:04,-14.737303,342.234404,-
3,2023-03-21,11:03,-2.040537,355.222681,-
4,2023-03-22,11:03,7.814569,10.088727,NEW MOON
5,2023-03-23,11:02,18.651085,23.026703,1 Ramadhan
6,2023-03-24,11:02,28.629482,35.752331,-
7,2023-03-25,11:01,37.874656,48.079556,-
8,2023-03-26,11:01,45.566347,60.012764,-
9,2023-03-27,11:00,51.541198,71.570404,-



CALCULATION FOR RAMADHAN 1444-09-01 -> 2023-03-23


*Forecast First Ied*

In [7]:
ijtimak_ied = forecastIjtimak(longitude, latitude, min_10_ied, plus_10_ied, '1 Syawwal')
print()
print('CALCULATION FOR IED', ied_hijri, '->', ijtimak_ied.strftime("%Y-%m-%d"))

,date,sunset,altitude,elongation,remark
0,2023-04-16,10:51,-40.635613,310.787018,-
1,2023-04-17,10:51,-30.885685,324.155907,-
2,2023-04-18,10:51,-20.424503,337.450443,-
3,2023-04-19,10:50,-9.420719,350.577213,-
4,2023-04-20,10:50,1.683452,3.512813,-
5,2023-04-21,10:50,12.016164,16.106550,NEW MOON
6,2023-04-22,10:49,22.401842,28.370140,1 Syawwal
7,2023-04-23,10:49,32.022724,40.296673,-
8,2023-04-24,10:49,40.876930,51.894337,-
9,2023-04-25,10:48,48.768227,63.197685,-



CALCULATION FOR IED 1444-10-01 -> 2023-04-22


In [8]:
ijtimak_delta = ijtimak_ied - ijtimak_ramadhan

print('EXPECTED RAMADHAN=', ramadhan, 'CALCULATION=', ijtimak_ramadhan.strftime("%Y-%m-%d"))
print('EXPECTED IED=', ied, 'CALCULATION=', ijtimak_ied.strftime("%Y-%m-%d"))
print()

print('RAMADHAN', HIJRI_YEAR, '=', ijtimak_ramadhan.strftime("%Y-%m-%d"))
print('IED', HIJRI_YEAR, '=', ijtimak_ied.strftime("%Y-%m-%d"))
print(f'Difference is {ijtimak_delta.days} days')

EXPECTED RAMADHAN= 2023-03-23 CALCULATION= 2023-03-23
EXPECTED IED= 2023-04-21 CALCULATION= 2023-04-22

RAMADHAN 1444 = 2023-03-23
IED 1444 = 2023-04-22
Difference is 30 days


*Forecasting Iedul Adha*

In [9]:
deltaTime = timedelta(days=5)

# 1 Dzulhijjah
dzulhijjah_hijri = Hijri(HIJRI_YEAR, 12, 1)
dzulhijjah = dzulhijjah_hijri.to_gregorian()
min_10_dzulhijjah = dzulhijjah - deltaTime
plus_10_dzulhijjah = dzulhijjah + deltaTime
print()
print('EXPECTED 1 DZULHIJJAH', dzulhijjah_hijri, '->', dzulhijjah)

ijtimak_dzulhijjah = forecastIjtimak(longitude, latitude, min_10_dzulhijjah, plus_10_dzulhijjah, '1 Dzulhijjah')
ijtimak_ied_adha = ijtimak_dzulhijjah + timedelta(days=9)
print()
print('CALCULATION FOR 1 DZULHIJJAH', dzulhijjah_hijri, '->', ijtimak_dzulhijjah.strftime("%Y-%m-%d"))
print('CALCULATION FOR 10 DZULHIJJAH', ijtimak_ied_adha.strftime("%Y-%m-%d"))


EXPECTED 1 DZULHIJJAH 1444-12-01 -> 2023-06-19


,date,sunset,altitude,elongation,remark
0,2023-06-14,10:46,-45.112850,315.782494,-
1,2023-06-15,10:46,-33.589002,327.817028,-
2,2023-06-16,10:46,-22.067264,339.558224,-
3,2023-06-17,10:46,-10.616976,350.705988,-
4,2023-06-18,10:46,1.111396,5.178372,-
5,2023-06-19,10:47,11.825894,15.030320,NEW MOON
6,2023-06-20,10:47,22.907877,25.914265,1 Dzulhijjah
7,2023-06-21,10:47,33.907678,36.805431,-
8,2023-06-22,10:47,44.828338,47.643932,-
9,2023-06-23,10:48,55.495659,58.465691,-



CALCULATION FOR 1 DZULHIJJAH 1444-12-01 -> 2023-06-20
CALCULATION FOR 10 DZULHIJJAH 2023-06-29
